In [1]:
import pandas as pd
import numpy as np

In [2]:
#Obtener la lista de los IDs de los pacientes:

# Cargar los archivos CSV en DataFrames
df1 = pd.read_csv('Diuresis.csv', encoding='ISO-8859-1', sep=';', decimal=',')
df2 = pd.read_csv('Perf.csv', encoding='ISO-8859-1', sep=';', decimal=',')
df3 = pd.read_csv('Datos grafica.csv', encoding='ISO-8859-1', sep=';', decimal=',')
df4 = pd.read_csv('Datos laboratorio.csv', encoding='ISO-8859-1', sep=';', decimal=',')
df5 = pd.read_csv('peso_data_patients.csv', encoding='ISO-8859-1', sep=';', decimal=',')

# Obtener todos los IDs de los cuatro DataFrames y agregarlos a una lista
ids = sorted(list(set(df1['idPaciente']).union(set(df2['idPaciente'])).union(set(df3['IdPaciente'])).union(set(df4['IdPaciente']))))
# Unir los cuatro DataFrames en uno solo
df = pd.concat([df1, df2, df3, df4], ignore_index=True)
# Eliminar filas duplicadas basadas en la columna "ID"
#df = df.drop_duplicates(subset=['IdPaciente'])
print('La lista de los IDs de los pacientes es:', ids)


# Diuresis.csv
# Obtener los IDs de los pacientes que han recibido los fármacos que buscamos
medications = ['Noradrenalina', 'Dobutamina']
perf_patients = df2[df2['PrincipioActivo'].isin(medications)]['idPaciente']
# Obtener los IDs de los pacientes que aparecen en ambos DataFrames
ids = sorted(list(set(df1.merge(df5, on='FechaIngresoUnidad', how='inner')['idPaciente']).intersection(set(perf_patients))))
# Calcular la diuresis en cc/kg/h para cada paciente y filtrar los que tienen una diuresis mayor a 0.5 cc/kg/h
for patient_id in ids:
    # Obtener la fecha de ingreso del paciente en Diuresis.csv
    diuresis_date = df1[df1['idPaciente'] == patient_id]['FechaIngresoUnidad'].values[0]
    # Obtener el peso del paciente en peso_data_patients.csv
    patient_weight = df5[df5['FechaIngresoUnidad'] == diuresis_date]['Peso'].values[0]
    # Filtrar las filas correspondientes al paciente actual
    patient_rows = df1[df1['idPaciente'] == patient_id]
    # Calcular la diuresis en cc/kg/h
    total_diuresis = patient_rows['Volumen'].sum()
    diuresis_rate = total_diuresis / patient_weight / len(patient_rows)
    if diuresis_rate > 0.5:
        print(f'El paciente {patient_id} tiene una diuresis de {diuresis_rate} cc/kg/h, que es mayor a 0.5 cc/kg/h')


        
# Perf.csv
# Convertir la columna "Valor" a valores numéricos
df2['Valor'] = pd.to_numeric(df2['Valor'], errors='coerce')
# Filtrar los pacientes que cumplen los criterios y obtener información de fecha y hora
filtered_data_norad = df2[(df2['PrincipioActivo'] == 'Noradrenalina') & (df2['Valor'] <= 0.1)]
filtered_data_dobu = df2[(df2['PrincipioActivo'] == 'Dobutamina') & (df2['Valor'] <= 5)]
filtered_data = pd.concat([filtered_data_norad, filtered_data_dobu], ignore_index=True)
filtered_data = filtered_data[['idPaciente', 'FechaRegistro', 'PrincipioActivo', 'Valor']]
# Ordenar por fecha y hora de registro
filtered_data = filtered_data.sort_values(['idPaciente', 'FechaRegistro'])
# Agrupar por paciente e imprimir información
for patient_id, patient_data in filtered_data.groupby('idPaciente'):
    print(f"Paciente {patient_id}:")
    for index, row in patient_data.iterrows():
        print(f"  - En {row['FechaRegistro']} el valor de {row['PrincipioActivo'].lower()} es {row['Valor']}")

        
        
# Datosgrafica.csv
# Convertir la columna "Valor" a valores numéricos
df3['Valor'] = pd.to_numeric(df3['Valor'], errors='coerce')
# Filtrar los pacientes que cumplen los criterios y obtener información de fecha y hora
filtered_data = df3[(df3['Propiedad'] == 'Medio') & (df3['Valor'] > 65)]
filtered_data = filtered_data[['IdPaciente', 'DíaHoraEntradaDatoEnGráfica', 'Valor']]
# Ordenar por fecha y hora de registro
filtered_data = filtered_data.sort_values(['IdPaciente', 'DíaHoraEntradaDatoEnGráfica'])
# Agrupar por paciente e imprimir información
for patient_id, patient_data in filtered_data.groupby('IdPaciente'):
    print(f"Paciente {patient_id}:")
    for index, row in patient_data.iterrows():
        print(f"  - En {row['DíaHoraEntradaDatoEnGráfica']} el valor de TA media es: {row['Valor']} mm Hg > 65 mm Hg")        
        

             
# Datoslaboratorio.csv
# Convertir la columna "Valor" a valores numéricos
df4['Valor'] = pd.to_numeric(df4['Valor'], errors='coerce')
# Filtrar los pacientes que cumplen los criterios y obtener información de fecha y hora
filtered_data = df4[(df4['Propiedad'] == 'Ácido láctico') & (df4['Valor'] < 1.8)]
filtered_data = filtered_data[['IdPaciente', 'DíaHoraEntradaDatoEnGráfica', 'Valor']]
# Ordenar por fecha y hora de registro
filtered_data = filtered_data.sort_values(['IdPaciente', 'DíaHoraEntradaDatoEnGráfica'])
# Agrupar por paciente e imprimir información
for patient_id, patient_data in filtered_data.groupby('IdPaciente'):
    print(f"Paciente {patient_id}:")
    for index, row in patient_data.iterrows():
        print(f"  - En {row['DíaHoraEntradaDatoEnGráfica']} el valor de ácido láctico: {row['Valor']} < 1'8")

La lista de los IDs de los pacientes es: ['0162AF3288ADF5AC6447B70B2E8E226E', '0272187BD95181C1D6DE533585787B21', '07CC4C9143AFB4ED41020310B231DD56', '107E0EEF53D121D2F3BCBDA1852E94F5', '11510E3994375CE0D0FC16FF4CA94389', '14AB461EF2D09C18B0454C4D330C30E7', '1ACA9AC915A39E669971BE0B88E0982D', '1B1290F17FB3818DEC4A0CDB80FB0995', '237195A5028E0817AC135AA06725ACB1', '23D4B6141F85D09DFD878F7C3C19E57F', '29AB34145B5D4AE58D01B9E1433C622E', '2ABF30AC03F710EDD0CEE90AB377417F', '30D3BE0DBEF040302EF2643E8711A27F', '316819B02E8BE9F3CDB2628D1BA8695F', '318F89F58A41F63ADAE331909D18554B', '36DC9A286351B752D0023C74FADEC158', '389580DF52D371AE68BC1EA79F3D0779', '3998C89CB276FB19A6B604F542CF8FB7', '3CA45D7EEF5BBCC18A98FF7789A41992', '3FA5931F669E95CEBCD0B6ACC88B7700', '44E590E5FB6F22AB1BFE0BDA3D6ADA9B', '45F4534C2DDF07FFDF4C4620C5E644FB', '47D47D666AF57D1C85D95A9803E75A73', '48FFE431532E6E8EBA33E1D9967B7383', '4B6CE1115FBFEE12F9015752D43154AD', '4C02C1465A01E86F40401DC6D7AF46E8', '50D5082FD92A4433AB001

Paciente 0162AF3288ADF5AC6447B70B2E8E226E:
  - En 2023-02-28 22:45:00.000 el valor de TA media es: 86.0 mm Hg > 65 mm Hg
  - En 2023-02-28 23:01:00.000 el valor de TA media es: 107.0 mm Hg > 65 mm Hg
  - En 2023-02-28 23:15:00.000 el valor de TA media es: 102.0 mm Hg > 65 mm Hg
  - En 2023-02-28 23:30:00.000 el valor de TA media es: 105.0 mm Hg > 65 mm Hg
  - En 2023-02-28 23:45:00.000 el valor de TA media es: 109.0 mm Hg > 65 mm Hg
  - En 2023-03-01 00:01:00.000 el valor de TA media es: 79.0 mm Hg > 65 mm Hg
  - En 2023-03-01 01:39:00.000 el valor de TA media es: 77.0 mm Hg > 65 mm Hg
  - En 2023-03-01 02:00:00.000 el valor de TA media es: 99.0 mm Hg > 65 mm Hg
  - En 2023-03-01 03:00:00.000 el valor de TA media es: 97.0 mm Hg > 65 mm Hg
  - En 2023-03-01 04:45:00.000 el valor de TA media es: 88.0 mm Hg > 65 mm Hg
  - En 2023-03-01 05:00:00.000 el valor de TA media es: 72.0 mm Hg > 65 mm Hg
  - En 2023-03-01 06:06:00.000 el valor de TA media es: 81.0 mm Hg > 65 mm Hg
  - En 2023-03-01

  - En 2023-01-27 07:30:00.000 el valor de TA media es: 79.0 mm Hg > 65 mm Hg
  - En 2023-01-27 08:01:00.000 el valor de TA media es: 67.0 mm Hg > 65 mm Hg
  - En 2023-01-27 09:00:00.000 el valor de TA media es: 86.0 mm Hg > 65 mm Hg
  - En 2023-01-27 09:31:00.000 el valor de TA media es: 77.0 mm Hg > 65 mm Hg
  - En 2023-01-27 10:01:00.000 el valor de TA media es: 76.0 mm Hg > 65 mm Hg
  - En 2023-01-27 10:31:00.000 el valor de TA media es: 79.0 mm Hg > 65 mm Hg
  - En 2023-01-27 11:01:00.000 el valor de TA media es: 111.0 mm Hg > 65 mm Hg
  - En 2023-01-27 11:31:00.000 el valor de TA media es: 96.0 mm Hg > 65 mm Hg
  - En 2023-01-27 12:01:00.000 el valor de TA media es: 87.0 mm Hg > 65 mm Hg
  - En 2023-01-27 12:31:00.000 el valor de TA media es: 80.0 mm Hg > 65 mm Hg
  - En 2023-01-27 13:01:00.000 el valor de TA media es: 86.0 mm Hg > 65 mm Hg
  - En 2023-01-27 13:31:00.000 el valor de TA media es: 84.0 mm Hg > 65 mm Hg
  - En 2023-01-27 14:01:00.000 el valor de TA media es: 119.0 m

  - En 2023-02-28 12:00:00.000 el valor de TA media es: 101.0 mm Hg > 65 mm Hg
  - En 2023-02-28 13:01:00.000 el valor de TA media es: 107.0 mm Hg > 65 mm Hg
  - En 2023-02-28 14:02:00.000 el valor de TA media es: 108.0 mm Hg > 65 mm Hg
  - En 2023-02-28 17:22:00.000 el valor de TA media es: 135.0 mm Hg > 65 mm Hg
  - En 2023-03-01 13:00:00.000 el valor de TA media es: 99.0 mm Hg > 65 mm Hg
  - En 2023-03-01 13:00:00.000 el valor de TA media es: 99.0 mm Hg > 65 mm Hg
  - En 2023-03-01 14:00:00.000 el valor de TA media es: 105.0 mm Hg > 65 mm Hg
  - En 2023-03-01 14:00:00.000 el valor de TA media es: 105.0 mm Hg > 65 mm Hg
  - En 2023-03-01 15:00:00.000 el valor de TA media es: 100.0 mm Hg > 65 mm Hg
  - En 2023-03-01 15:00:00.000 el valor de TA media es: 100.0 mm Hg > 65 mm Hg
  - En 2023-03-01 16:01:00.000 el valor de TA media es: 97.0 mm Hg > 65 mm Hg
  - En 2023-03-01 17:00:00.000 el valor de TA media es: 98.0 mm Hg > 65 mm Hg
  - En 2023-03-01 18:00:00.000 el valor de TA media es: 

  - En 2023-01-29 17:13:00.000 el valor de TA media es: 89.0 mm Hg > 65 mm Hg
  - En 2023-01-29 17:43:00.000 el valor de TA media es: 86.0 mm Hg > 65 mm Hg
  - En 2023-01-29 18:13:00.000 el valor de TA media es: 82.0 mm Hg > 65 mm Hg
  - En 2023-01-29 18:43:00.000 el valor de TA media es: 88.0 mm Hg > 65 mm Hg
  - En 2023-01-29 19:13:00.000 el valor de TA media es: 87.0 mm Hg > 65 mm Hg
  - En 2023-01-29 19:43:00.000 el valor de TA media es: 92.0 mm Hg > 65 mm Hg
  - En 2023-01-29 20:13:00.000 el valor de TA media es: 91.0 mm Hg > 65 mm Hg
  - En 2023-01-29 20:43:00.000 el valor de TA media es: 104.0 mm Hg > 65 mm Hg
  - En 2023-01-29 21:01:00.000 el valor de TA media es: 94.0 mm Hg > 65 mm Hg
  - En 2023-01-29 21:13:00.000 el valor de TA media es: 88.0 mm Hg > 65 mm Hg
  - En 2023-01-29 21:43:00.000 el valor de TA media es: 90.0 mm Hg > 65 mm Hg
  - En 2023-01-29 22:13:00.000 el valor de TA media es: 87.0 mm Hg > 65 mm Hg
  - En 2023-01-29 22:43:00.000 el valor de TA media es: 88.0 mm

  - En 2023-01-13 22:54:00.000 el valor de TA media es: 78.0 mm Hg > 65 mm Hg
  - En 2023-01-13 23:54:00.000 el valor de TA media es: 81.0 mm Hg > 65 mm Hg
  - En 2023-01-14 00:54:00.000 el valor de TA media es: 96.0 mm Hg > 65 mm Hg
  - En 2023-01-14 01:54:00.000 el valor de TA media es: 95.0 mm Hg > 65 mm Hg
  - En 2023-01-14 02:54:00.000 el valor de TA media es: 103.0 mm Hg > 65 mm Hg
  - En 2023-01-14 03:54:00.000 el valor de TA media es: 101.0 mm Hg > 65 mm Hg
  - En 2023-01-14 04:54:00.000 el valor de TA media es: 87.0 mm Hg > 65 mm Hg
  - En 2023-01-14 05:54:00.000 el valor de TA media es: 105.0 mm Hg > 65 mm Hg
  - En 2023-01-14 06:54:00.000 el valor de TA media es: 77.0 mm Hg > 65 mm Hg
  - En 2023-01-14 07:53:00.000 el valor de TA media es: 82.0 mm Hg > 65 mm Hg
  - En 2023-01-14 08:55:00.000 el valor de TA media es: 80.0 mm Hg > 65 mm Hg
  - En 2023-01-14 10:17:00.000 el valor de TA media es: 86.0 mm Hg > 65 mm Hg
  - En 2023-01-14 10:54:00.000 el valor de TA media es: 99.0 

  - En 2023-02-16 01:16:00.000 el valor de TA media es: 67.0 mm Hg > 65 mm Hg
  - En 2023-02-16 01:31:00.000 el valor de TA media es: 70.0 mm Hg > 65 mm Hg
  - En 2023-02-16 01:46:00.000 el valor de TA media es: 69.0 mm Hg > 65 mm Hg
  - En 2023-02-16 02:00:00.000 el valor de TA media es: 66.0 mm Hg > 65 mm Hg
  - En 2023-02-16 02:16:00.000 el valor de TA media es: 78.0 mm Hg > 65 mm Hg
  - En 2023-02-16 02:31:00.000 el valor de TA media es: 74.0 mm Hg > 65 mm Hg
  - En 2023-02-16 02:46:00.000 el valor de TA media es: 69.0 mm Hg > 65 mm Hg
  - En 2023-02-16 03:00:00.000 el valor de TA media es: 69.0 mm Hg > 65 mm Hg
  - En 2023-02-16 03:31:00.000 el valor de TA media es: 70.0 mm Hg > 65 mm Hg
  - En 2023-02-16 04:01:00.000 el valor de TA media es: 71.0 mm Hg > 65 mm Hg
  - En 2023-02-16 04:31:00.000 el valor de TA media es: 73.0 mm Hg > 65 mm Hg
  - En 2023-02-16 05:00:00.000 el valor de TA media es: 78.0 mm Hg > 65 mm Hg
  - En 2023-02-16 05:31:00.000 el valor de TA media es: 70.0 mm 

Paciente 0162AF3288ADF5AC6447B70B2E8E226E:
  - En 2023-03-03 19:46:00.000 el valor de ácido láctico: 1.1 < 1'8
  - En 2023-03-04 20:45:00.000 el valor de ácido láctico: 1.6 < 1'8
  - En 2023-03-05 20:44:00.000 el valor de ácido láctico: 1.7 < 1'8
Paciente 0272187BD95181C1D6DE533585787B21:
  - En 2023-02-15 19:45:00.000 el valor de ácido láctico: 1.0 < 1'8
  - En 2023-02-16 11:46:00.000 el valor de ácido láctico: 0.9 < 1'8
  - En 2023-02-17 08:47:00.000 el valor de ácido láctico: 1.7 < 1'8
  - En 2023-02-18 18:44:00.000 el valor de ácido láctico: 1.4 < 1'8
Paciente 07CC4C9143AFB4ED41020310B231DD56:
  - En 2023-02-26 21:45:00.000 el valor de ácido láctico: 0.5 < 1'8
  - En 2023-02-27 08:46:00.000 el valor de ácido láctico: 1.1 < 1'8
  - En 2023-03-01 08:46:00.000 el valor de ácido láctico: 1.3 < 1'8
  - En 2023-03-03 07:44:00.000 el valor de ácido láctico: 0.7 < 1'8
  - En 2023-03-07 07:44:00.000 el valor de ácido láctico: 0.9 < 1'8
  - En 2023-03-09 08:45:00.000 el valor de ácido láctic

In [10]:
# Filtrar los pacientes que cumplen todos los parámetros
filtered_patients = []
for patient_id in ids:
    # Filtro del archivo Diuresis.csv
    diur_data = df1[(df1['idPaciente'] == patient_id) & (diuresis_rate > 0.5)]
    if len(diur_data) == 0:
        continue
        

    # Filtro del archivo Perf.csv
    perf_data_norad = df2[(df2['idPaciente'] == patient_id) & (df2['PrincipioActivo'] == 'Noradrenalina') & (df2['Valor'] <= 0.1)]
    perf_data_dobu = df2[(df2['idPaciente'] == patient_id) & (df2['PrincipioActivo'] == 'Dobutamina') & (df2['Valor'] <= 5)]
    if len(perf_data_norad) == 0 or len(perf_data_dobu) == 0:
        continue
    
    # Filtro del archivo Datosgrafica.csv
    graf_data = df3[(df3['IdPaciente'] == patient_id) & (df3['Propiedad'] == 'Medio') & (df3['Valor'] > 65)]
    if len(graf_data) == 0:
        continue
    
    # Filtro del archivo Datoslaboratorio.csv
    lab_data = df4[(df4['IdPaciente'] == patient_id) & (df4['Propiedad'] == 'Ácido láctico') & (df4['Valor'] < 1.8)]
    if len(lab_data) == 0:
        continue
    
    # Si el paciente cumple todos los criterios, se agrega a la lista de pacientes filtrados
    filtered_patients.append(patient_id)

print('La lista de los IDs de los pacientes que cumplen todos los parámetros es:', filtered_patients)

La lista de los IDs de los pacientes que cumplen todos los parámetros es: ['52F37BDE8E5AAA15C9AA55339497914F', '5CA0C79FEDAB60E8599337D676A2C978']


In [ ]:
#DASHBOARD

In [4]:
import streamlit as st
import pandas as pd

# Cargar los archivos CSV en DataFrames
df1 = pd.read_csv('Diuresis.csv', encoding='ISO-8859-1', sep=';', decimal=',')
df2 = pd.read_csv('Perf.csv', encoding='ISO-8859-1', sep=';', decimal=',')
df3 = pd.read_csv('Datos grafica.csv', encoding='ISO-8859-1', sep=';', decimal=',')
df4 = pd.read_csv('Datos laboratorio.csv', encoding='ISO-8859-1', sep=';', decimal=',')
df5 = pd.read_csv('peso_data_patients.csv', encoding='ISO-8859-1', sep=';', decimal=',')

# Obtener todos los IDs de los cuatro DataFrames y agregarlos a una lista
ids = sorted(list(set(df1['idPaciente']).union(set(df2['idPaciente'])).union(set(df3['IdPaciente'])).union(set(df4['IdPaciente']))))

# Unir los cuatro DataFrames en uno solo
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

# Definir límite por defecto
DEFAULT_LIMIT = 10

# Desplegable de IDs de pacientes
patient_id = st.selectbox('Selecciona un paciente:', ids)

# Filtrar datos del paciente seleccionado
filtered_data = df[df['idPaciente'] == patient_id]

# Mostrar información básica del paciente
patient_info = filtered_data[['idPaciente', 'FechaRegistro', 'Edad']].iloc[0]
st.write('Información del paciente:')
st.write(patient_info)

# Filtros para valores específicos
min_diuresis = st.sidebar.number_input('Diuresis mínima', value=0.5)
max_lactico = st.sidebar.number_input('Ácido láctico máximo', value=1.8)
min_medio = st.sidebar.number_input('Medio mínimo', value=65)
max_norad = st.sidebar.number_input('Noradrenalina máxima', value=0.1)
max_dobu = st.sidebar.number_input('Dobutamina máxima', value=5)

# Aplicar filtros
filtered_data = filtered_data[(filtered_data[''] > min_diuresis) &
                              (filtered_data['Valor'] < max_lactico) &
                              (filtered_data['Valor'] > min_medio) &
                              (filtered_data['PrincipioActivo'] == 'Noradrenalina') &
                              (filtered_data['Valor'] <= max_norad) &
                              (filtered_data['PrincipioActivo'] == 'Dobutamina') &
                              (filtered_data['Valor'] <= max_dobu)]

# Mostrar tabla de datos del paciente seleccionado
st.write('Tabla de datos del paciente:')
st.write(filtered_data[['Evento', 'Propiedad', 'Valor', 'Fecha', 'Warning', 'Alerta']])

# Verificar si se cumplen todos los parámetros
if len(filtered_data) > 0:
    st.success('El paciente cumple todos los parámetros')
else:
    st.warning('El paciente no cumple todos los parámetros')


2023-04-10 12:54:44.953 
  command:

    streamlit run C:\PYTHON\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2023-04-10 12:54:45.003 Serialization of dataframe to Arrow table was unsuccessful due to: ("Expected bytes, got a 'numpy.int64' object", 'Conversion failed for column 6 with type object'). Applying automatic fixes for column types to make the dataframe Arrow-compatible.


KeyError: ''